In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import zipfile
import re
import csv 

In [2]:
PERIODO = '202406'
NUM_PERIODO = 202406
PERIODO_DX = '202406'

## Stock Móvil

In [3]:
columnas_seleccionadas = ["RUT",
                          "SUB_CLAS_CRP",
                          "ESTADO_CONTRATO",
                          "PCS",
                          "CRP",
                          "REG_ENV_CRP",
                          'COMUNA_LEG_CRP',
                          "PLAN_ACTUAL",
                          "ANTIGUEDAD_CONTRATO",
                          "ANTIGUEDAD_PLAN"
                         ,"GRUPO_CLIENTE_VANTIVE"]

In [4]:
cartera_movil= pd.read_csv('..\..\MOVIL\Stocks\CLARO\TABLA E\CARTERA_POSTPAGO_'+PERIODO+'_'+PERIODO +'.txt', sep="|", usecols=columnas_seleccionadas, encoding="ISO-8859-1")  # modificar

In [5]:
cartera_movil['PERIODO_STOCK'] = NUM_PERIODO

In [6]:
cartera_movil['ESTADO_CONTRATO'].value_counts()

ACTIVA                            3325991
FINALIZADO                         171700
SUSPENDIDA                          14652
Suspensión Collection (Subscri       2029
Name: ESTADO_CONTRATO, dtype: int64

Dejar solo segmento Persona y PCS Activos

In [7]:
# cartera_movil = cartera_movil.loc[cartera_movil['ESTADO_CONTRATO'] == 'ACTIVA']  # dejar solo activos (revisar)
cartera_movil = cartera_movil.loc[cartera_movil['SUB_CLAS_CRP'] == 'PERSONA']  # dejar solo b2c 

In [8]:
cartera_movil = cartera_movil.loc[cartera_movil['ESTADO_CONTRATO'] == 'ACTIVA']

In [9]:
cartera_movil['ESTADO_CONTRATO'].value_counts()

ACTIVA    2437802
Name: ESTADO_CONTRATO, dtype: int64

In [10]:
## Eliminar duplicados
#cartera_movil = cartera_movil.drop_duplicates(subset=['PCS','CRP'])

In [11]:
# Crear un nuevo campo 'Q_LINEAS' con el conteo de las veces que se repite cada RUT
cartera_movil['Q_LINEAS'] = cartera_movil.groupby('RUT')['RUT'].transform('count')

In [12]:
#Excluir funcionarios
pattern = r'PLAN FUNCIONARIO|EMPLEADO'
cartera_movil = cartera_movil[~cartera_movil['GRUPO_CLIENTE_VANTIVE'].str.contains(pattern, case=False, na=False)]


In [13]:
# Crear el nuevo campo 'MANDATO'
cartera_movil['MANDATO'] = np.where(cartera_movil['PLAN_ACTUAL'].str.contains('GGTT', case=False, na=False), 1, 0)

In [14]:
cartera_movil['MANDATO'].value_counts()

0    2289677
1     145574
Name: MANDATO, dtype: int64

In [15]:
# Filtrar para eliminar registros donde 'PLAN_ACTUAL' contiene 'GGTT' (sin importar mayúsculas/minúsculas)
cartera_movil = cartera_movil[~cartera_movil['PLAN_ACTUAL'].str.contains('GGTT', case=False, na=False)]

In [16]:
cartera_movil.shape

(2289677, 14)

##  Cartera Móvil Teorica


In [17]:
import pandas as pd

# Definir la ruta del archivo correctamente
archivo = r"C:\Users\ctorof\CLAROCHILE\Revenue Management, CBM y Calidad de Venta - Documentos\Revenue Móvil\Archivos Claro Móvil\Stock ARPU Teórico Claro Móvil\Stock Teorico 202406 Claro Móvil.txt"

cartera_teorica = pd.read_csv(archivo, delimiter='\t')  # Asumiendo que el archivo está delimitado por tabulaciones

# Renombrar la columna BA_NO a CRP
cartera_teorica = cartera_teorica.rename(columns={'BA_NO': 'CRP'})


# Imprimir el contenido del DataFrame
print(cartera_teorica)




                 CRP           PCS       ARPU_CF      ARPU_PYD          ARPU
0          3000008.0  5.699840e+10   7554.621849  -3777.319328   3777.302521
1          3000009.0  5.697670e+10  12596.638655 -10102.504202   2494.134454
2          3000019.0  5.699865e+10   9487.394958  -2531.218487   6956.176471
3          3000026.0  5.695472e+10   7736.134454  -1933.983193   5802.151261
4          3000026.0  5.697780e+10  31924.369748      0.000000  31924.369748
...              ...           ...           ...           ...           ...
2760319  245874966.0  5.696897e+10  12596.638655      0.000000  12596.638655
2760320  245875185.0  5.697288e+10  12596.638655      0.000000  12596.638655
2760321  245875488.0  5.699003e+10  10915.966387      0.000000  10915.966387
2760322  245875494.0  5.698754e+10  10915.966387      0.000000  10915.966387
2760323  245876568.0  5.694708e+10  12596.638655      0.000000  12596.638655

[2760324 rows x 5 columns]


In [18]:
#### Correcciones (correcciones hasta la base de mayo 2024)

#cartera_facturada[0][0] = cartera_facturada[0][0].replace("ï»¿", "")

# Definir Nombre de Columnas"camnia ba_no
#cartera_facturada.columns = ['PCS','CRP','ARPU','ARPU_CF','ARPU_PYD'] 

In [19]:
cartera_teorica['PCS'].fillna(value=0, inplace=True)
cartera_teorica['CRP'].fillna(value=0, inplace=True)
cartera_teorica['PCS'] = cartera_teorica['PCS'].astype(float)
cartera_teorica['CRP'] = cartera_teorica['CRP'].astype(float)

#### Corregir  Datos Arpu

cartera_teorica['ARPU'] = cartera_teorica['ARPU'].astype(str)
cartera_teorica['ARPU'] = cartera_teorica['ARPU'].str.replace(',', '.')
cartera_teorica['ARPU'] = cartera_teorica['ARPU'].astype(float)

cartera_teorica['ARPU_CF'] = cartera_teorica['ARPU_CF'].astype(str)
cartera_teorica['ARPU_CF'] = cartera_teorica['ARPU_CF'].str.replace(',', '.')
cartera_teorica['ARPU_CF'] = cartera_teorica['ARPU_CF'].astype(float)

cartera_teorica['ARPU_PYD'] = cartera_teorica['ARPU_PYD'].astype(str)
cartera_teorica['ARPU_PYD'] = cartera_teorica['ARPU_PYD'].str.replace(',', '.')
cartera_teorica['ARPU_PYD'] = cartera_teorica['ARPU_PYD'].astype(float)

In [20]:
# Ajusta el formato de la columna 'PCS' y 'CRP' en cartera_facturada -> Dejarlo como la base de cartera

cartera_teorica['PCS'] = cartera_teorica['PCS'].apply(lambda x: '{:.0f}'.format(float(x)) if pd.notna(x) else x)
cartera_teorica['CRP'] = cartera_teorica['CRP'].apply(lambda x: '{:.0f}'.format(float(x)) if pd.notna(x) else x)

# Elimina duplicados en cartera_facturada basándote en las columnas 'PCS', 'CRP'
cartera_teorica = cartera_teorica.drop_duplicates(subset=['PCS','CRP'])

In [21]:
cartera_teorica.isnull().sum()

CRP         0
PCS         0
ARPU_CF     0
ARPU_PYD    0
ARPU        0
dtype: int64

##### Unir Cartera [Stock] con Cartera [Arpu Facturado]

In [22]:
#    Dejar todos los campos en el mismo formato

cartera_teorica['PCS'] = pd.to_numeric(cartera_teorica['PCS'], errors='coerce')
cartera_teorica['CRP'] = pd.to_numeric(cartera_teorica['CRP'], errors='coerce')

cartera_movil['PCS'] = pd.to_numeric(cartera_movil['PCS'], errors='coerce')
cartera_movil['CRP'] = pd.to_numeric(cartera_movil['CRP'], errors='coerce')

In [23]:
Tabla_E_Movil_Claro = pd.merge(cartera_movil, cartera_teorica , on=['PCS','CRP'], how='left')
#Tabla_E_Movil_Claro['ARPU'].fillna('arpu no encontrado',inplace=True)

In [24]:
# ver = cartera_movil1[cartera_movil1['ARPU'] == 'arpu no encontrado']
# ver

In [25]:
# cartera_movil1['ARPU'].value_counts()

In [26]:
Tabla_E_Movil_Claro.isnull().sum()

PCS                         0
RUT                         0
REG_ENV_CRP                 1
COMUNA_LEG_CRP             35
CRP                         0
SUB_CLAS_CRP                0
GRUPO_CLIENTE_VANTIVE       0
PLAN_ACTUAL                 0
ESTADO_CONTRATO             0
ANTIGUEDAD_CONTRATO         0
ANTIGUEDAD_PLAN             0
PERIODO_STOCK               0
Q_LINEAS                    0
MANDATO                     0
ARPU_CF                  6168
ARPU_PYD                 6168
ARPU                     6168
dtype: int64

In [27]:
#resumen = cartera_movil1.groupby(['ARPU','ESTADO_CONTRATO']).agg(Q=('PCS', 'count')).reset_index()
#resumen 

#### Rango de antiguedad

In [28]:
Tabla_E_Movil_Claro['AÑOS'] = Tabla_E_Movil_Claro['ANTIGUEDAD_CONTRATO']/12

Tabla_E_Movil_Claro.loc[Tabla_E_Movil_Claro['AÑOS'] < 0.5, 'RANGO_ANTIGUEDAD'] = '[< 6 meses]'
Tabla_E_Movil_Claro.loc[np.logical_and(Tabla_E_Movil_Claro['AÑOS'] >= 0.5, Tabla_E_Movil_Claro['AÑOS'] < 2),'RANGO_ANTIGUEDAD'] = '[6 meses- 2 año]'
Tabla_E_Movil_Claro.loc[Tabla_E_Movil_Claro['AÑOS'] >= 2, 'RANGO_ANTIGUEDAD'] = '[> 2 años]'

In [29]:
Tabla_E_Movil_Claro.isnull().sum()

PCS                         0
RUT                         0
REG_ENV_CRP                 1
COMUNA_LEG_CRP             35
CRP                         0
SUB_CLAS_CRP                0
GRUPO_CLIENTE_VANTIVE       0
PLAN_ACTUAL                 0
ESTADO_CONTRATO             0
ANTIGUEDAD_CONTRATO         0
ANTIGUEDAD_PLAN             0
PERIODO_STOCK               0
Q_LINEAS                    0
MANDATO                     0
ARPU_CF                  6168
ARPU_PYD                 6168
ARPU                     6168
AÑOS                        0
RANGO_ANTIGUEDAD            0
dtype: int64

In [30]:
Tabla_E_Movil_Claro# dejar hasta aca 

,PCS,RUT,REG_ENV_CRP,COMUNA_LEG_CRP,CRP,SUB_CLAS_CRP,GRUPO_CLIENTE_VANTIVE,PLAN_ACTUAL,ESTADO_CONTRATO,ANTIGUEDAD_CONTRATO,ANTIGUEDAD_PLAN,PERIODO_STOCK,Q_LINEAS,MANDATO,ARPU_CF,ARPU_PYD,ARPU,AÑOS,RANGO_ANTIGUEDAD
0,56982511936,18367656-3,13.0,PUENTE ALTO,236279388,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,22,22,202406,1,0,14799.159664,-4439.756303,10359.403361,1.833333,[6 meses- 2 año]
1,56998452152,12341930-8,13.0,COLINA,229217447,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,37,37,202406,5,0,16315.966387,-3263.201681,13052.764706,3.083333,[> 2 años]
2,56993989188,19903514-2,13.0,BUIN,229473527,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,36,36,202406,2,0,14277.310924,-8566.386555,5710.924370,3.000000,[> 2 años]
3,56953580722,11865909-0,13.0,LO BARNECHEA,237115763,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,20,20,202406,1,0,15355.462185,-7677.781513,7677.680672,1.666667,[6 meses- 2 año]
4,56965373952,12882304-2,13.0,PUENTE ALTO,233128268,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,29,29,202406,5,0,14845.378151,-5938.084034,8907.294118,2.416667,[> 2 años]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289672,56954200632,12697784-0,8.0,SAN PEDRO DE LA PAZ,216497679,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM LIBRE,ACTIVA,63,63,202406,4,0,20728.571429,0.000000,20728.571429,5.250000,[> 2 años]
2289673,56981345847,9224242-0,2.0,ANTOFAGASTA,234430177,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM LIBRE,ACTIVA,26,26,202406,2,0,19196.638655,-7246.865546,11949.773109,2.166667,[> 2 años]
2289674,56940486352,10881848-4,13.0,HUECHURABA,230985586,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM LIBRE,ACTIVA,32,32,202406,2,0,12596.638655,-3778.991597,8817.647059,2.666667,[> 2 años]
2289675,56964957572,16967216-4,14.0,VALDIVIA,235886021,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM BLACK LIBRE,ACTIVA,23,23,202406,3,0,24104.201681,-6026.159664,18078.042017,1.916667,[6 meses- 2 año]


In [31]:
# Rellenar valores NA en la columna ARPU_CF con 0
Tabla_E_Movil_Claro['ARPU'] = Tabla_E_Movil_Claro['ARPU'].fillna(0)

# Convertir los valores de la columna ARPU_CF a enteros
Tabla_E_Movil_Claro['ARPU'] = Tabla_E_Movil_Claro['ARPU'].astype(int)

In [32]:
Tabla_E_Movil_Claro

,PCS,RUT,REG_ENV_CRP,COMUNA_LEG_CRP,CRP,SUB_CLAS_CRP,GRUPO_CLIENTE_VANTIVE,PLAN_ACTUAL,ESTADO_CONTRATO,ANTIGUEDAD_CONTRATO,ANTIGUEDAD_PLAN,PERIODO_STOCK,Q_LINEAS,MANDATO,ARPU_CF,ARPU_PYD,ARPU,AÑOS,RANGO_ANTIGUEDAD
0,56982511936,18367656-3,13.0,PUENTE ALTO,236279388,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,22,22,202406,1,0,14799.159664,-4439.756303,10359,1.833333,[6 meses- 2 año]
1,56998452152,12341930-8,13.0,COLINA,229217447,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,37,37,202406,5,0,16315.966387,-3263.201681,13052,3.083333,[> 2 años]
2,56993989188,19903514-2,13.0,BUIN,229473527,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,36,36,202406,2,0,14277.310924,-8566.386555,5710,3.000000,[> 2 años]
3,56953580722,11865909-0,13.0,LO BARNECHEA,237115763,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,20,20,202406,1,0,15355.462185,-7677.781513,7677,1.666667,[6 meses- 2 año]
4,56965373952,12882304-2,13.0,PUENTE ALTO,233128268,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,29,29,202406,5,0,14845.378151,-5938.084034,8907,2.416667,[> 2 años]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289672,56954200632,12697784-0,8.0,SAN PEDRO DE LA PAZ,216497679,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM LIBRE,ACTIVA,63,63,202406,4,0,20728.571429,0.000000,20728,5.250000,[> 2 años]
2289673,56981345847,9224242-0,2.0,ANTOFAGASTA,234430177,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM LIBRE,ACTIVA,26,26,202406,2,0,19196.638655,-7246.865546,11949,2.166667,[> 2 años]
2289674,56940486352,10881848-4,13.0,HUECHURABA,230985586,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM LIBRE,ACTIVA,32,32,202406,2,0,12596.638655,-3778.991597,8817,2.666667,[> 2 años]
2289675,56964957572,16967216-4,14.0,VALDIVIA,235886021,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM BLACK LIBRE,ACTIVA,23,23,202406,3,0,24104.201681,-6026.159664,18078,1.916667,[6 meses- 2 año]


In [33]:
# Definir los límites y las etiquetas para los rangos
bins = [float('-inf'), 3001, 4001, 5001, 6001, 7001, 8001, 9001, 10001, 11001, 12001, 13001, 14001, 15001, 16001, 17001, 18001, 19001, 20001, float('inf')]
labels = ['0-3000', '3000-4000', '4000-5000', '5000-6000', '6000-7000', '7000-8000', '8000-9000', '9000-10000', '10000-11000', '11000-12000', '12000-13000', '13000-14000', '14000-15000', '15000-16000', '16000-17000', '17000-18000', '18000-19000', '19000-20000', '+20000']

# Crear la columna Rango_ARPU_CF utilizando pd.cut()
Tabla_E_Movil_Claro['Rango_ARPU'] = pd.cut(Tabla_E_Movil_Claro['ARPU'], bins=bins, labels=labels, right=True)

In [34]:
# Filtrar los registros donde Q_LINEAS sea igual a 1
tabla_E_movil_claro_filtrada = Tabla_E_Movil_Claro#[Tabla_E_Movil_Claro['Q_LINEAS'] == 1]

In [35]:
tabla_E_movil_claro_filtrada.RUT.value_counts()

19560580-7    34
14107062-2    31
16660926-7    16
23808391-5    16
24385997-2    15
              ..
14594363-9     1
17631831-7     1
19755893-8     1
6198496-8      1
13995079-8     1
Name: RUT, Length: 1494322, dtype: int64

In [36]:
tabla_E_movil_claro_filtrada

,PCS,RUT,REG_ENV_CRP,COMUNA_LEG_CRP,CRP,SUB_CLAS_CRP,GRUPO_CLIENTE_VANTIVE,PLAN_ACTUAL,ESTADO_CONTRATO,ANTIGUEDAD_CONTRATO,ANTIGUEDAD_PLAN,PERIODO_STOCK,Q_LINEAS,MANDATO,ARPU_CF,ARPU_PYD,ARPU,AÑOS,RANGO_ANTIGUEDAD,Rango_ARPU
0,56982511936,18367656-3,13.0,PUENTE ALTO,236279388,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,22,22,202406,1,0,14799.159664,-4439.756303,10359,1.833333,[6 meses- 2 año],10000-11000
1,56998452152,12341930-8,13.0,COLINA,229217447,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,37,37,202406,5,0,16315.966387,-3263.201681,13052,3.083333,[> 2 años],13000-14000
2,56993989188,19903514-2,13.0,BUIN,229473527,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,36,36,202406,2,0,14277.310924,-8566.386555,5710,3.000000,[> 2 años],5000-6000
3,56953580722,11865909-0,13.0,LO BARNECHEA,237115763,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,20,20,202406,1,0,15355.462185,-7677.781513,7677,1.666667,[6 meses- 2 año],7000-8000
4,56965373952,12882304-2,13.0,PUENTE ALTO,233128268,PERSONA,Normal Cliente,RP 05 2021-ARR-MAX XL LIBRE,ACTIVA,29,29,202406,5,0,14845.378151,-5938.084034,8907,2.416667,[> 2 años],8000-9000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289672,56954200632,12697784-0,8.0,SAN PEDRO DE LA PAZ,216497679,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM LIBRE,ACTIVA,63,63,202406,4,0,20728.571429,0.000000,20728,5.250000,[> 2 años],+20000
2289673,56981345847,9224242-0,2.0,ANTOFAGASTA,234430177,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM LIBRE,ACTIVA,26,26,202406,2,0,19196.638655,-7246.865546,11949,2.166667,[> 2 años],11000-12000
2289674,56940486352,10881848-4,13.0,HUECHURABA,230985586,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM LIBRE,ACTIVA,32,32,202406,2,0,12596.638655,-3778.991597,8817,2.666667,[> 2 años],8000-9000
2289675,56964957572,16967216-4,14.0,VALDIVIA,235886021,PERSONA,Normal Cliente,RP 05 2021-EP-MAX PREMIUM BLACK LIBRE,ACTIVA,23,23,202406,3,0,24104.201681,-6026.159664,18078,1.916667,[6 meses- 2 año],18000-19000


In [37]:
DF = pd.read_csv(r"C:\Users\ctorof\Downloads\RESUMEN_CONSUMO_GB_MARZO_MAY 1.txt", sep=",", encoding='latin-1')

In [38]:
DF

,Unnamed: 0,PCS,NEGOCIO,CONSUMO_TOTAL_GB_MARZO,CONSUMO_TOTAL_GB_ABR,CONSUMO_TOTAL_GB_MAY,PROMEDIO_CONSUMO,CONSUMO_CATEGORIA
0,0,56950544396,POSTPAGO,12.065847,11.588061,25.761196,16.471701,< 40 GB
1,1,56950544453,POSTPAGO,272.313751,62.532809,108.879393,147.908651,>= 40 GB
2,2,56950544843,POSTPAGO,0.000000,0.001036,0.227739,0.076258,< 40 GB
3,3,56950546290,POSTPAGO,22.314457,33.535181,NaN,27.924819,< 40 GB
4,4,56950546576,POSTPAGO,130.207021,177.985512,186.635927,164.942820,>= 40 GB
...,...,...,...,...,...,...,...,...
2893983,2893983,56999921031,POSTPAGO,NaN,NaN,0.000000,0.000000,< 40 GB
2893984,2893984,56999950343,POSTPAGO,NaN,NaN,6.410573,6.410573,< 40 GB
2893985,2893985,56999955806,POSTPAGO,NaN,NaN,0.570363,0.570363,< 40 GB
2893986,2893986,56999989429,POSTPAGO,NaN,NaN,1.772892,1.772892,< 40 GB


In [38]:
tabla_E_movil_claro_filtrada=tabla_E_movil_claro_filtrada.merge(DF[['PCS','PROMEDIO_CONSUMO','CONSUMO_CATEGORIA']],on=['PCS'], how='left')

In [39]:
tabla_E_movil_claro_filtrada['CONSUMO_CATEGORIA'].fillna('< 40 GB',inplace=True)

In [40]:
tabla_E_movil_claro_filtrada['CATEGORIA_Q_LINEAS']=np.where(tabla_E_movil_claro_filtrada['Q_LINEAS']==1,'1 Linea','+1 Linea')

In [42]:
Score_DX=pd.read_csv(r"C:\Users\ctorof\Downloads\delivery_dt=2024-05-31_volChurn_proactive_score_2024-05-31.csv")

In [44]:
Score_DX['SCORE_CATEGORIA']=np.where((Score_DX.score > 0.9),'0.9 a 1', 
                                     np.where((Score_DX.score > 0.8),'0.8 a 0.9',
                                              np.where((Score_DX.score > 0.7),'0.7 a 0.8','0 a 0.7')))

In [45]:
tabla_E_movil_claro_filtrada = tabla_E_movil_claro_filtrada.merge(Score_DX, how = 'left', on = 'PCS')
tabla_E_movil_claro_filtrada['SCORE_CATEGORIA'].fillna('SIN CATEGORIA', inplace = True)

In [46]:
x = tabla_E_movil_claro_filtrada.groupby(['Rango_ARPU','RANGO_ANTIGUEDAD','CONSUMO_CATEGORIA',
                                      'CATEGORIA_Q_LINEAS','SCORE_CATEGORIA']).agg(ARPU_PROMEDIO=('ARPU','mean'),
                                                                                   ARPU_TOTAL=('ARPU','sum'),
                                                                                   Q_CLIENTES=('PCS','count'),
                                                                                   Q_LINEAS_PROM=('Q_LINEAS','mean')).reset_index()#.to_excel(r"C:\Users\wduran\CLAROCHILE\Revenue Management, CBM y Calidad de Venta - Documentos\CBM\MOVIL\Stocks\Resumen_Segmento_CLARO_MOVIL.xlsx",index=False)

In [47]:
x = x.loc[x.Q_CLIENTES != 0]
x.to_excel(r"C:\Users\ctorof\CLAROCHILE\Revenue Management, CBM y Calidad de Venta - Documentos\CBM\MOVIL\Stocks\Resumen_Segmento_CLARO_MOVIL.xlsx",index=False)